In [1]:
import pygame
import random
#import _thread
from copy import deepcopy
import numpy as np



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.font.init()
Pause=False
Eat = 0
Cherry = [10, 10]

In [3]:
class snake():
    def __init__(self,x,y):
        self.live=600
        self.done=False 
        self.reward=0.0
        self.pep=1
        self.x=x
        self.y=y
        self.HSpeed = 1
        self.VSpeed = 0
        self.Snake = [[x, y],[x-1, y],[x-2, y]]
        
    def CreateCherry(self):
        global Cherry
        random.seed()
        Y = random.randint(2, 17)
        X = random.randint(2, 17)
        Cherry = [X, Y]
        for i in self.Snake:
            if i == Cherry:
                self.CreateCherry()
                return
    def EatCherry(self,Cherry1):
        if Cherry1 == self.Snake[0]:
            self.CreateCherry()
            return True
        else:
            return False
        
    def Move(self):      
        self.done=False  
        diff1=abs(self.Snake[0][0]-Cherry[0])+abs(self.Snake[0][1]-Cherry[1])
        self.Head = deepcopy(self.Snake[0])
        self.Snake[0][0] += self.HSpeed 
        self.Snake[0][1] += self.VSpeed
        self.reward=0.0
        diff2=abs(self.Snake[0][0]-Cherry[0])+abs(self.Snake[0][1]-Cherry[1])
        if diff2<diff1:
            self.live-=1
            if self.pep>0:
                self.pep+=1.0
            else:
                self.pep=1.0
            self.reward=min(4.0,0.4*self.pep)
        else:
            if self.pep<0:
                self.pep-=1.0
            else:
                self.pep=-1.0
            self.live-=2
            self.reward=max(-8.0,0.8*self.pep)
        for i in range(1, len(self.Snake)):
            self.Snake[i], self.Head = self.Head, self.Snake[i]        
        if self.EatCherry(Cherry)==True:
            self.reward+=50.0
            self.live+=100
            self.Snake.append(self.Head)            
    

    
            
    def restart(self):
        self.live-=20
        if self.live<0:
            if self.live>-50:
                self.reward-=10.0
            else:
                self.reward+=10
            self.done=True
            self.live=600
        self.reward-=10.0
        self.HSpeed = 1
        self.VSpeed = 0
        tr=random.randint(0, 4)
        self.Snake = [[self.x+tr, self.y],[self.x+tr-1, self.y],[self.x+tr-2, self.y]]
    
    def Check(self):
        if self.live<=0:
            self.restart()
            return
        for i in range(1, len(self.Snake)):
            if self.Snake[0] == self.Snake[i]:
                self.restart()
                return
        
        if self.Snake[0][0] > 19 or self.Snake[0][0] < 0:
            self.restart()
            return
        if self.Snake[0][1] > 19 or self.Snake[0][1] < 0:
            self.restart()
            return
    def external_Check(other_snake,self):
        if self.Snake[0] == other_snake.Snake[0]:
                other_snake.restart()
                self.restart()
                return
        for i in range(1, len(other_snake.Snake)):
            if self.Snake[0] == other_snake.Snake[i]:
                self.restart()
                return

In [4]:
class Graphics(pygame.sprite.Sprite):
    def __init__(self, win):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((800, 800))
        self.image.fill((0, 0, 0))
        self.rect = self.image.get_rect()
        self.rect.center = (400, 400)
    def update(self, win,snek):
        global Cherry
        self.image = pygame.Surface((800, 800))
        self.image.fill((0, 255, 0))
        for p in range(len(snek)):
            for k in snek[p].Snake:
                pygame.draw.rect(self.image, (255-p*(255//len(snek)), p*(255//len(snek)), 0), ((k[0]*40, k[1]*40), (40,40)) )                    
        pygame.draw.rect(self.image, (255, 0, 255), ((Cherry[0]*40, Cherry[1]*40), (40,40)) )
        
        
class Menu(pygame.sprite.Sprite):
    def __init__(self, win):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((800, 800))
        self.image.fill((0, 0, 0))
        self.rect = self.image.get_rect()
        self.rect.center = (400, 400)
    def update(self, win):
        global Cherry
        self.image = pygame.Surface((800, 800))
        self.image.fill((0, 255, 0))
        pygame.draw.rect(self.image, (255, 0, 0), ((200, 200), (400,200)) )                    
        pygame.draw.rect(self.image, (0, 0, 255), ((200, 500), (400,200)) )
        fnt = pygame.font.SysFont("arial", 40)
        text = fnt.render(('RESUME'), 1, (0, 0, 0))
        self.image.blit(text, (400-text.get_rect().width//2, 300-text.get_rect().height//2))       

In [5]:
Visual = True

In [6]:
if Visual == True:
    pygame.init()
    pygame.mixer.init()
    screen = pygame.display.set_mode((800, 800))
    clock = pygame.time.Clock()
    Spritesgame = pygame.sprite.Group()
    Spritespause = pygame.sprite.Group()
    Spritesgame.add(Graphics(screen))
    Spritespause.add(Menu(screen))

In [7]:
snek=[snake(5,5),snake(5,15)]

In [8]:
def getDeck():
    global Cherry, snek
    Deck = []
    a=[0]*20
    for i in range(20):
        Deck.append(a[::])

    Deck[Cherry[0]][Cherry[1]] = 10
    for i in range(len(snek)):
        Deck[snek[i].Snake[0][0]][snek[i].Snake[0][1]] = (i+2)*2
        for j in range(1,len(snek[i].Snake)):
            Deck[snek[i].Snake[j][0]][snek[i].Snake[j][1]] = (i+2)*2-1
    return np.asarray(Deck).ravel()
Desk=getDeck()

In [9]:
def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def swish(x):
    return x * sigmoid(x)

def swish_der(x):
    return  swish(x) + sigmoid(x)* (1 - swish(x))

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

class Agent(object):
    def __init__(self):
        self.memory={}
        self.memory['xs']=[]
        self.memory['hs']=[]
        self.memory['dlogps']=[]
        self.memory['drs']=[]
        self.gamma=0.8
        self.lr=0.0001
        self.decay=0.99
        self.batch=16 
        self.l1n=300
        self.l2n=64
        self.Dim=20*20
        self.G=0
        self.model={}
        self.model['W1'] = np.random.randn(self.l1n,self.Dim) / np.sqrt(self.Dim) # "Xavier" initialization
        self.model['W2'] = np.random.randn(3,self.l1n) / np.sqrt(self.l1n)

        self.grad_buffer = { k : np.zeros_like(v) for k,v in self.model.items() } #
        self.rmsprop_cache = { k : np.zeros_like(v) for k,v in self.model.items() } # rmsprop memory

        
        
    def dics_rewards(self,reward_memory):
        G = np.zeros_like(reward_memory)
        for pen in range(len(reward_memory)):
            for t in range(len(reward_memory[pen])):
                G_sum = 0
                discount = 1
                if reward_memory[pen][t]>10:
                    for k in range(1, t):
                        if reward_memory[pen][t-k]<-3:
                            break
                        reward_memory[pen][t-k]+=reward_memory[pen][t] * discount


                        discount *= self.gamma
                    
                G[pen][t] = reward_memory[pen][t]
        norm = max(G.max(),G.min(), key=abs)
        G = G / norm
        return G

    def policy_forward(self,x):
        h = np.dot(self.model['W1'], x)
        h=swish(h) # ReLU nonlinearity
        logp = np.dot(self.model['W2'], h)
        p = softmax(logp)
        return p,h

    def policy_backward(self,eph, epdlogp, epx):
        """ backward pass. (eph is array of intermediate hidden states) """
        #print(eph.T.shape,'eph')
        #print(epdlogp.shape,'epdlogp')
        dW2 = np.matmul(eph.T, epdlogp).T
        #print(dW2.shape,'dW2')
        dh = np.matmul(epdlogp, self.model['W2'])
        #print(dh.shape,'dh')
        dh = swish_der(dh) # backpro prelu
        dW1 = np.matmul(dh.T, epx)
        #print(dW1.shape,'dW1')
        return {'W1':dW1, 'W2':dW2}

    def remember(self,deck,hidden,reward,grad):
        self.memory['xs'].append(deck)
        self.memory['hs'].append(hidden)
        self.memory['dlogps'].append(grad)
        self.memory['drs'].append(reward)

    def choose_action(self, p):
        p=[p[0]/(p[0]+p[1]+p[2]),p[1]/(p[0]+p[1]+p[2]),p[2]/(p[0]+p[1]+p[2])]
        action = np.random.choice(3,1,p=p)
        return action
    def load_model(self,filename):
        npzfile = np.load(filename)
        self.model['W1']=npzfile['arr_0']
        self.model['W2']=npzfile['arr_1']

    def learn(self,epnum):
        epx = np.vstack(self.memory['xs'])
        eph = np.vstack(self.memory['hs'])
        epdlogp = np.vstack(self.memory['dlogps'])
        
        epr = np.vstack(self.memory['drs'])
        #print(epr)
        
        self.memory['xs']=[]
        self.memory['hs']=[]
        self.memory['dlogps']=[]
        self.memory['drs']=[]

        disc_epr=self.dics_rewards(epr)
        #print(disc_epr)
        #print(disc_epr.shape)
        #print(epdlogp.shape)
        for i in range(len(epdlogp)):
            #for t in epdlogp[i]:
                #print(t.shape)
                #print(disc_epr[i].shape)
            epdlogp[i]*=disc_epr[i]
        grad = self.policy_backward(eph, epdlogp, epx)
        for k in self.model: self.grad_buffer[k] += grad[k]
        if epnum%self.batch==0:
            for k,v in self.model.items():
                g = self.grad_buffer[k] # gradient
                self.rmsprop_cache[k] = self.decay * self.rmsprop_cache[k] + (1 - self.decay) * g**2
                self.model[k] += self.lr * g / (np.sqrt(self.rmsprop_cache[k]) + 1e-5)
                self.grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        


    


    

In [10]:
learning=True
sai1= Agent()
sai2= Agent()
sai1.load_model( 'data11.npz')
sai1.load_model( 'data22.npz')


In [11]:
step=0
score1=[]
score2=[]
score_history1=[]
score_history2=[]
move_history1=[]
move_history2=[]
tiknum=0
emnum=0

In [12]:
runing = True

while runing:
    if learning== True:
        p1,h1 = sai1.policy_forward(Desk)
        #print(p.sum())
        move1= sai1.choose_action(p1)
        move_history1.append(move1)
        if len(move_history1)>11:
            move_history1.pop(0)
        if move_history1[-4:]==[1,1,1,1]: 
            move1=0
            move_history1[-1]=0
        if move_history1[-9:]==[2,2,2,2,2,2,2,2,2]: 
            move1=1
            move_history1[-1]=1
        if move_history1[-4:]==[0,0,0,0]: 
            move1=2
            move_history1[-1]=2
        if move1==0:
            snek[0].HSpeed, snek[0].VSpeed =  snek[0].VSpeed, -snek[0].HSpeed 
        elif move1==1:
            snek[0].HSpeed, snek[0].VSpeed =  -snek[0].VSpeed, snek[0].HSpeed
        grad1=[0,0,0]
        for i in range(len(grad1)):
            if i==move1:
                grad1[i]=1-p1[i]
            else:
                grad1[i]=-p1[i]

        p2,h2 = sai2.policy_forward(Desk)
        #print(p.sum())
        move2= sai2.choose_action(p2)
        move_history2.append(move2)
        if len(move_history2)>11:
            move_history2.pop(0)
        if move_history2[-4:]==[1,1,1,1]: 
            move2=0
            move_history2[-1]=0
        if move_history2[-9:]==[2,2,2,2,2,2,2,2,2]: 
            move2=1
            move_history2[-1]=1
        if move_history2[-4:]==[0,0,0,0]: 
            move2=2
            move_history2[-1]=2
        if move2==0:
            snek[1].HSpeed, snek[1].VSpeed =  snek[1].VSpeed, -snek[1].HSpeed 
        elif move2==1:
            snek[1].HSpeed, snek[1].VSpeed =  -snek[1].VSpeed, snek[1].HSpeed
        grad2=[0,0,0]
        for i in range(len(grad2)):
            if i==move2:
                grad2[i]=1-p2[i]
            else:
                grad2[i]=-p2[i]
                



        snek[1].Move()
        snek[0].Move()
        snek[1].Check()
        snek[0].Check()
        snek[0].external_Check(snek[1])
        snek[1].external_Check(snek[0])
        if snek[0].done==True and snek[1].done==True:
            pass
        elif snek[0].done==True:
            snek[1].live=-1
            snek[1].reward+=20
            snek[1].restart()
        elif snek[1].done==True:
            snek[0].live=-1
            snek[0].reward+=20
            snek[0].restart()

        reward1=snek[0].reward
        reward2=snek[1].reward
        sai1.remember(Desk,h1,reward1,grad1)
        sai2.remember(Desk,h2,reward2,grad2)
        oldDesk=Desk
        Desk=getDeck()

        score1.append(snek[0].reward)
        score2.append(snek[1].reward)
        if snek[0].done==True:
            emnum+=1
            sai1.learn(emnum)
            sai2.learn(emnum)
        #print(snek[1].reward)
        #print(snek[0].reward)
        snek[1].reward=0
        snek[0].reward=0
        
        if snek[0].done==True:
            score_history1.append(sum(1 for y in score1 if y > 10))
            score_history2.append(sum(1 for y in score2 if y > 10))

            tiknum+=1
            if tiknum%50==1:
                np.savez( 'data11.npz', sai1.model['W1'],sai1.model['W2'])
                np.savez( 'data22.npz', sai2.model['W1'],sai2.model['W2'])
                

                for i in range(20):
                    print(oldDesk[i*20:i*20+20])
                print(tiknum, np.mean(score1),np.mean(score_history1[-100:]))
                print(tiknum, np.mean(score2),np.mean(score_history2[-100:]))
            score1=[]
            score2=[]
            if tiknum%400000==1:
                pygame.init()
                pygame.mixer.init()
                screen = pygame.display.set_mode((800, 800))
                clock = pygame.time.Clock()
                Spritesgame = pygame.sprite.Group()
                Spritespause = pygame.sprite.Group()
                Spritesgame.add(Graphics(screen))
                Spritespause.add(Menu(screen))
                Visual=True


        
    if Visual == True:
        clock.tick(15)
        for event in pygame.event.get():
            # check for closing window
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                pygame.quit()
                Visual=False
                '''
                if Pause == True:
                    Pause = False
                else:
                    Pause = True
                '''
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_BACKSPACE:
                runing = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_RIGHT:
                if snek[0].HSpeed == 0:
                    snek[0].HSpeed = 1
                    snek[0].VSpeed = 0

            elif event.type == pygame.KEYDOWN and event.key == pygame.K_LEFT:
                if snek[0].HSpeed == 0:
                    snek[0].HSpeed = -1
                    snek[0].VSpeed = 0
            
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                if snek[0].VSpeed == 0:
                    snek[0].HSpeed = 0
                    snek[0].VSpeed = -1
            
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                if snek[0].VSpeed == 0:
                    snek[0].HSpeed = 0
                    snek[0].VSpeed = 1
            
            ########################################################################################################################
            
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_d:
                if snek[1].HSpeed == 0:
                    snek[1].HSpeed = 1
                    snek[1].VSpeed= 0

            elif event.type == pygame.KEYDOWN and event.key == pygame.K_a:
                if snek[1].HSpeed == 0:
                    snek[1].HSpeed = -1
                    snek[1].VSpeed = 0
            
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_w:
                if snek[1].VSpeed == 0:
                    snek[1].HSpeed = 0
                    snek[1].VSpeed = -1
            
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_s:
                if snek[1].VSpeed == 0:
                    snek[1].HSpeed = 0
                    snek[1].VSpeed = 1
            #####################################################################################################################################
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pose = pygame.mouse.get_pos()
                if Pause == True:
                    if (pose[0] > 200) and (pose[0] < 600) and (pose[1] < 400) and (pose[1] > 200):
                        Pause = False
                
        
    if Pause == False: 
        if learning== False:               
            snek[1].Move()
            snek[0].Move()
            snek[1].Check()
            snek[0].Check()
            snek[0].external_Check(snek[1])
            snek[1].external_Check(snek[0])
            if snek[0].done==True and snek[1].done==True:
                snek[1].live=-1
                snek[1].restart()
            elif snek[0].done==True:
                snek[1].live=-100
                snek[1].reward+=6
                snek[1].restart()
            elif snek[1].done==True:
                snek[0].live=-100
                snek[0].reward+=6
                snek[0].restart()
        if Visual == True:
            Spritesgame.update(screen,snek)
            Spritesgame.draw(screen)
    else:
        if Visual == True:
            Spritespause.update(screen)
            Spritespause.draw(screen)
    if Visual == True:
        pygame.display.flip()
if Visual == True:    
    pygame.quit()

0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0]
[0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 6 0 0 0 0]
[0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
260401 -0.6432432432432433 0.0
260401 -4.4270270270270276 0.0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0]
[0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 5 0 0 0 0]

KeyboardInterrupt: 

In [13]:
pygame.quit()